In [1]:
import torch
import torch.nn as nn
import pickle
import requests
import numpy as np
from scipy.stats import spearmanr

import sys
import os
sys.path.append(os.path.abspath('../..'))
from app.classes.glove_model import Glove

In [2]:
with open('../../app/models/glove/glove.pkl', 'rb') as f:
    model = pickle.load(f)

with open('../../app/models/glove/glove_word2index.pkl', 'rb') as f:
    word2index = pickle.load(f)

with open('../../app/models/glove/glove_index2word.pkl', 'rb') as f:
    index2word = pickle.load(f)

center_embeddings = model.center_embedding.weight.data
outside_embeddings = model.outside_embedding.weight.data

In [3]:
def get_embed(word):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.center_embedding(id_tensor)
    u_embed = model.outside_embedding(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [4]:
def get_dot_product(a, b):
    dot_product = np.dot(a, b)
    return dot_product

In [5]:
with open('../helper/wordsim_similarity_goldstandard.txt', 'r') as f:
    content = f.readlines()

lines = [line.strip().replace('\t', ' ').split() for line in content]

In [6]:
similarity = []

for line in lines:
    word1 = line[0]
    word2 = line[1]
    score = float(line[2])

    try:
        similarity.append(get_dot_product(get_embed(word1), get_embed(word2)))
    except:
        similarity.append(0.0)

# find the Spearman Correlation
spearman_correlation = spearmanr(similarity, [line[2] for line in lines])
print(spearman_correlation)

# find the MSE
squared_error = [(similarity[i] - float(lines[i][2]))**2 for i in range(len(similarity))]
mse = sum(squared_error) / len(similarity)
print(mse)

SignificanceResult(statistic=-0.012778188159983711, pvalue=0.8564113053521886)
32.99920552368159
